<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/agent/react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Agent with Query Engine (RAG) Tools

In this section, we show how to setup an agent powered by the ReAct loop for financial analysis.

The agent has access to two "tools": one to query the 2021 Lyft 10-K and the other to query the 2021 Uber 10-K.

We try two different LLMs:

- gpt-3.5-turbo
- gpt-3.5-turbo-instruct

Note that you can plug in any LLM that exposes a text completion endpoint.

## Build Query Engine Tools

In [1]:
# %pip install llama-index-llms-openai

In [2]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [3]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/kbank"
    )
    kbank_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/scb"
    )
    scb_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/siri"
    )
    siri_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/ttb"
    )
    ttb_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

Download Data

In [4]:
# !mkdir -p 'data/10k/'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [6]:
from llama_index.llms.gemini import Gemini

llm = Gemini(model="models/gemini-pro")

In [9]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
from llama_index.core import Settings

# global
Settings.embed_model = embed_model
Settings.llm = llm
Settings.chunk_size = 1024
Settings.chunk_overlap = 128

In [11]:
if not index_loaded:
    # load data
    kbank_docs = SimpleDirectoryReader(
        input_files=["kbank.PDF"]
    ).load_data()
    scb_docs = SimpleDirectoryReader(
        input_files=["scb.pdf"]
    ).load_data()
    siri_docs = SimpleDirectoryReader(
        input_files=["siri.PDF"]
    ).load_data()
    ttb_docs = SimpleDirectoryReader(
        input_files=["ttb.pdf"]
    ).load_data()

    # build index
    kbank_index = VectorStoreIndex.from_documents(kbank_docs)
    scb_index = VectorStoreIndex.from_documents(scb_docs)
    siri_index = VectorStoreIndex.from_documents(siri_docs)
    ttb_index = VectorStoreIndex.from_documents(ttb_docs)

    # persist index
    kbank_index.storage_context.persist(persist_dir="./storage/kbank")
    scb_index.storage_context.persist(persist_dir="./storage/scb")
    siri_index.storage_context.persist(persist_dir="./storage/siri")
    ttb_index.storage_context.persist(persist_dir="./storage/ttb")

In [35]:
kbank_engine = kbank_index.as_query_engine(similarity_top_k=10,node_postprocessors=[rerank])
scb_engine = scb_index.as_query_engine(similarity_top_k=10,node_postprocessors=[rerank])
siri_engine = siri_index.as_query_engine(similarity_top_k=10,node_postprocessors=[rerank])
ttb_engine = ttb_index.as_query_engine(similarity_top_k=10,node_postprocessors=[rerank])

## Setup ReAct Agent

Here we setup two ReAct agents: one powered by standard gpt-3.5-turbo, and the other powered by gpt-3.5-turbo-instruct.

You can **optionally** specify context which will be added to the core ReAct system prompt.

In [73]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [75]:
# [Optional] Add Context
# context = """\
# You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
#     You will answer questions about Uber and Lyft as in the persona of a sorcerer \
#     and veteran stock market investor.
# """
# llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose='debug',
    context=context
)

In [85]:
response = agent.chat("เปรียบเทียบนโยบายด้านสิ่งแวดล้อมของ scb, kbank, ttb")
print(str(response))

Thought: The current language of the user is: Thai. I need to use a tool to help me answer the question.
Action: scb_agent
Action Input: {'input': 'นโยบายด้านสิ่งแวดล้อม'}
Observation: ด�าเนินงานด้วยความใส่ใจสิ่งแวดล้อม ด้วยการลดผลกระทบต่อสิ่งแวดล้อมจากกิจกรรมทางธุรกิจ ควบคู่ไปกับการใช้ทรัพยากรธรรมชาติอย่างมีประสิทธิภาพและประสิทธิผลตามแนวทาง 3R ดังนี้
• Reduce การลดปริมาณการใช้ทรัพยากร 
• Reuse การน�าทรัพยากรที่ใช้แล้วกลับมาใช้ซ�้า  
• Recycle การน�าทรัพยากรกลับมาใช้ใหม่โดยผ่าน  
   กระบวนการรีไซเคิล 
ด�าเนินงานเพื่อปกป้องสิ่งแวดล้อม โดยปฏิบัติตามมาตรฐานและแนวปฏิบัติที่ดีที่สุดของอุตสาหกรรม ที่จะท�าให้เกิดการใช้ทรัพยากรธรรมชาติอย่างมีประสิทธิภาพ ลดการใช้พลังงาน  ลดการปล่อยก๊าซเรือนกระจก และลดการเกิดของเสีย รวมถึงการหาแนวทางปรับปรุงประสิทธิภาพการใช้พลังงาน ข้อก�าหนดเฉพาะกลุ่มอุตสาหกรรม (หน้า 414)
Thought: The current language of the user is: Thai. I need to use a tool to help me answer the question.
Action: kbank_agent
Action Input: {'input': 'นโยบายด้านสิ่งแวดล้อม'}
Observation: ธนาคาร

## Run Some Example Queries

We run some example queries using the agent, showcasing some of the agent's abilities to do chain-of-thought-reasoning and tool use to synthesize the right answer.

We also show queries.

In [256]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: kbank_agent
Action Input: {'input': 'revenue growth'}


KeyboardInterrupt: 

**Async execution**: Here we try another query with async execution

In [ ]:
# Try another query with async execution

import nest_asyncio

nest_asyncio.apply()

response = await agent.achat(
    "Compare and contrast the risks of Uber and Lyft in 2021, then give an"
    " analysis"
)
print(str(response))

Thought: (Implicit) I can answer without any more tools!
Answer: I'm sorry, but I currently don't have the tools to provide information about Uber and Lyft. I can only provide information about kbank and SCB.
I'm sorry, but I currently don't have the tools to provide information about Uber and Lyft. I can only provide information about kbank and SCB.


#### Taking a look at a turbo-instruct agent

In [ ]:
llm_instruct = OpenAI(model="gpt-3.5-turbo-instruct")
agent_instruct = ReActAgent.from_tools(
    query_engine_tools, llm=llm_instruct, verbose=True
)

In [ ]:
response = agent_instruct.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue growth in 2021 was 36%.
Response: Lyft's revenue growth in 2021 was 36%.
Lyft's revenue growth in 2021 was 36%.


#### Try more complex queries

We compare gpt-3.5-turbo with gpt-3.5-turbo-instruct agents on more complex queries.

In [ ]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Thought: I need to use a tool to help me compare the revenue growth of Uber and Lyft in 2021.
Action: uber_10k
Action Input: {'input': "Please provide information about Uber's revenue growth in 2021."}
Observation: Uber's revenue grew by 57% in 2021 compared to the previous year. This growth was primarily driven by an increase in Gross Bookings, with Delivery Gross Bookings increasing by 71% and Mobility Gross Bookings growing by 38%. The increase in Delivery Gross Bookings was due to higher demand for food delivery orders and expansion across U.S. and international markets. The growth in Mobility Gross Bookings was a result of increased Trip volumes as the business recovered from the impacts of COVID-19.
Thought: I have information about Uber's revenue growth in 2021. Now I need to use a tool to get information about Lyft's revenue growth in 2021.
Action: lyft_10k
Action Input: {'input': "Please provide information about Lyft's revenue growth in 2021."}
Observation: Lyft's revenue inc

In [ ]:
response = agent_instruct.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Response: The revenue growth of Uber was higher than Lyft in 2021, with Uber experiencing a 74% growth compared to Lyft's 48%. This indicates that Uber may have had a stronger financial performance in 2021. However, further analysis is needed to fully understand the factors contributing to this difference.
The revenue growth of Uber was higher than Lyft in 2021, with Uber experiencing a 74% growth compared to Lyft's 48%. This indicates that Uber may have had a stronger financial performance in 2021. However, further analysis is needed to fully understand the factors contributing to this difference.


In [ ]:
response = agent.chat(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

Thought: I need to find out which company has higher revenue before I can provide information about its risk factors.
Action: lyft_10k
Action Input: {'input': 'What is the revenue of Lyft in 2021?'}
Observation: The revenue of Lyft in 2021 is $3,208,323,000.
Thought: Now that I know Lyft has higher revenue, I can find information about its risk factors.
Action: lyft_10k
Action Input: {'input': 'What are the risk factors of Lyft?'}
Observation: Lyft faces numerous risk factors that could potentially harm its business, financial condition, and results of operations. These risk factors include general economic factors such as the impact of the COVID-19 pandemic, natural disasters, economic downturns, and political crises. Operational factors such as limited operating history, financial performance, competition, unpredictability of results, uncertainty regarding market growth, ability to attract and retain drivers and riders, insurance coverage, autonomous vehicle technology, reputation an

In [ ]:
response = agent_instruct.query(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

Response: The risk factors for the company with the higher revenue include competition, regulatory changes, and dependence on drivers.
The risk factors for the company with the higher revenue include competition, regulatory changes, and dependence on drivers.


**Observation**: The turbo-instruct agent seems to do worse on agent reasoning compared to the regular turbo model. Of course, this is subject to further observation!